In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import palettable
import math

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

import lightgbm as lgb
import shap

In [ ]:
df = pd.read_excel("./data/用于问题23的数据.xlsx")

In [ ]:
df.columns

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

sns.set_theme(style="white", palette="husl")
plt.figure(dpi=1000)
x = np.array(df['5 or more tries (X)'])
y = np.array(df['percentage_proc'])

z1 = np.polyfit(x, y, 1)              # 曲线拟合，返回值为多项式的各项系数
p1 = np.poly1d(z1)                    # 返回值为多项式的表达式，也就是函数式子
print(p1)
y_pred = p1(x)                        # 根据函数的多项式表达式，求解 y
# print(np.polyval(p1, 29))             根据多项式求解特定 x 对应的 y 值
# print(np.polyval(z1, 29))             根据多项式求解特定 x 对应的 y 值

plot1 = plt.plot(x, y, '.b', label='original values')
plot2 = plt.plot(x, y_pred, 'r', label='fit values')
plt.title('')
plt.xlabel('')
plt.ylabel('')
plt.legend(loc=3, borderaxespad=0., bbox_to_anchor=(0, 0))
plt.show()

In [ ]:
df["5 or more tries (X)"] = df['5 tries'] + df['6 tries'] + df['7 or more tries (X)']
sns.set()
sns.lmplot(data=df, x='5 or more tries (X)', y='percentage_proc')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

sns.set_theme(style="white", palette="husl")
plt.figure(dpi=1000)
x = np.array(df['4 or less tries (X)'])
y = np.array(df['percentage_proc'])

z1 = np.polyfit(x, y, 1)              # 曲线拟合，返回值为多项式的各项系数
p1 = np.poly1d(z1)                    # 返回值为多项式的表达式，也就是函数式子
print(p1)
y_pred = p1(x)                        # 根据函数的多项式表达式，求解 y
# print(np.polyval(p1, 29))             根据多项式求解特定 x 对应的 y 值
# print(np.polyval(z1, 29))             根据多项式求解特定 x 对应的 y 值

plot1 = plt.plot(x, y, '.b', label='original values')
plot2 = plt.plot(x, y_pred, 'r', label='fit values')
plt.title('')
plt.xlabel('')
plt.ylabel('')
plt.legend(loc=3, borderaxespad=0., bbox_to_anchor=(0, 0))
plt.show()

In [ ]:
df["4 or less tries (X)"] = df['1 try'] + df['2 tries'] + df['3 tries'] + df['4 tries']
sns.set()
sns.lmplot(data=df, x='4 or less tries (X)', y='percentage_proc')

In [ ]:
df.columns

In [ ]:
df.loc[:,["4 or less tries (X)", "5 or more tries (X)", "percentage_proc"]].head()

In [ ]:
df.columns

In [ ]:
# 难度定义，越难指标越高
# 4次以上、困难模式占比  --> 成本型
# 5次以上              --> 效益型
# 重复字母个数          --> 效益型
# 单词常见程度          --> 成本型
X = df.loc[:,["4 or less tries (X)", "5 or more tries (X)", "percentage_proc",
              'number of repeated letters', 'word commonness', ]].values
X[:, 2] = X[:, 2] * 1000
# X[:, 3] = np.exp(X[:, 3])


In [ ]:
max(X[:, 2]) - min(X[:, 2])

In [ ]:
# 数据规范化
X[:, 0] = (max(X[:, 0]) - X[:, 0]) / (max(X[:, 0]) - min(X[:, 0]))
X[:, 1] = (X[:, 1] - min(X[:, 1])) / (max(X[:, 1]) - min(X[:, 1]))
X[:, 2] = (max(X[:, 2]) - X[:, 2]) / (max(X[:, 2]) - min(X[:, 2]))
X[:, 3] = (X[:, 3] - min(X[:, 3])) / (max(X[:, 3]) - min(X[:, 3]))
X[:, 4] = (max(X[:, 4]) - X[:, 4]) / (max(X[:, 4]) - min(X[:, 4]))
# X[:, 3] = (max(X[:, 3]) - X[:, 3]) / (max(X[:, 3]) - min(X[:, 3]))

In [ ]:
X[11, 0] = 0
X[11, 1] = 0
X[23, 2] = 0
# X[:, 2]

In [ ]:
# 归一化矩阵
norm_X = X / np.sum(X, axis=0)
norm_X.shape

In [ ]:
# np.where(norm_X[:,3] == 0)[0]

In [ ]:
# norm_X.min()

In [ ]:
# 有元素为0，需要加一个极小值，使其大于0
norm_X = norm_X + 0.00001

In [ ]:
norm_X.shape[0]

In [ ]:
# 计算信息熵
entropy = -np.sum(norm_X * np.log(norm_X), axis=0) / np.log(norm_X.shape[0])
entropy

In [ ]:
# 计算权重
m = len(entropy)
sum_entropy = np.sum(entropy)
weight = (1 - entropy) / (m - sum_entropy)
weight

In [ ]:
X.shape

In [ ]:
z = np.matmul(X, weight)

In [ ]:
df["difficulty factor"] = z
# df['norm_4-'] = X[:,0]

In [ ]:
sns.set()
sns.lmplot(data=df, x='4 or less tries (X)', y='difficulty factor')

In [ ]:
sns.set()
sns.lmplot(data=df, x='5 or more tries (X)', y='difficulty factor')

In [ ]:
df.shape

In [ ]:
X.shape

In [ ]:
import numpy as np
from sklearn.datasets import make_blobs

def l1_weighted_dist(x, y, w):
    return np.sum(w * np.abs(x - y))

def weighted_euclidean_distance(x, y, w):
    return np.sqrt(np.sum(w * (x - y)**2))

def kmeans(X, k, w, max_iter=100):
    n_samples, n_features = X.shape

    # Step 1: Initialize k centroids randomly
    centroids = X[np.random.choice(n_samples, k, replace=False), :]

    # Step 2: Repeat until convergence or max iterations reached
    for i in range(max_iter):
        # Step 2.1: Assign samples to nearest centroid
        cluster_labels = np.zeros(n_samples)
        for j, x in enumerate(X):
            dists = [weighted_euclidean_distance(x, c, w) for c in centroids]
            cluster_labels[j] = np.argmin(dists)

        # Step 2.2: Update centroids
        for c in range(k):
            X_cluster = X[cluster_labels == c, :]
            if len(X_cluster) > 0:
                centroids[c, :] = np.average(X_cluster, axis=0)

    return centroids, cluster_labels

# Run k-means clustering with weighted L1 distance
centroids, labels = kmeans(X, k=3, w=weight)

# Print the centroids and cluster labels
print("Centroids:\n", centroids)
print("Cluster labels:\n", labels)


In [ ]:
max_k = 10  # 最大簇数
sse = []
silhouette_coeffs = []
for k in range(1, max_k + 1):
    labels, centers = kmeans(X, k, weight)
    # sse.append(np.sum([(X[i] - centers[int(labels[i])]) ** 2 for i in range(len(X))]))
    if k > 1:
        silhouette_coeffs.append(np.mean([silhouette_coefficient(X[i], labels[i], centers, weight) for i in range(len(X))]))

# 绘制肘部图
# plt.plot(range(1, max_k + 1), sse)
# plt.xlabel('Number of clusters')
# plt.ylabel('SSE')
# plt.title('Elbow Method')
# plt.show()

# 绘制轮廓系数
plt.plot(range(2, max_k + 1), silhouette_coeffs)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')

In [ ]:
# 定义肘部法则函数
def kmeans_sse(X, k, w):
    centroids, clusters = kmeans(X, k, w)
    sse = np.sum([np.sum((X[clusters==i] - centroids[i])**2) for i in range(k)])
    return sse

ks = range(1, 10) # 尝试的不同簇数
sses = [kmeans_sse(X, k, weight) for k in ks] # 计算每个簇数对应的SSE值
plt.plot(ks, sses, '-o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Sum of Squared Errors (SSE)')
plt.show()

In [ ]:
df['labels'] = labels
df.to_excel('data/data for 3.xlsx')

In [ ]:
df = pd.read_excel('data/data for 3.xlsx')

In [ ]:
print(df['difficulty factor'][df[df['labels'].apply(lambda x: x == 0)].index].mean())
print(df['difficulty factor'][df[df['labels'].apply(lambda x: x == 1)].index].mean())
print(df['difficulty factor'][df[df['labels'].apply(lambda x: x == 2)].index].mean())
# print(df['difficulty factor'][df[df['labels'].apply(lambda x: x == 3)].index].mean())
# print(df['difficulty factor'][df[df['labels'].apply(lambda x: x == 4)].index].mean())

In [ ]:
# sns.violinplot(data=df, x="labels", y="difficulty factor", split=True)
sns.set_theme(style="white", palette="pastel")
plt.figure(dpi=1000)
sns.boxplot(data=df, x="labels", y="difficulty factor", 
            showfliers=False, color='white', boxprops = {'color':'red',#箱子外框
           'facecolor':'pink'},
           showmeans=True,#箱图显示均值，
            meanline=True,#显示均值线
            meanprops = {'linestyle':'--','color':'red'},
            palette=palettable.tableau.TrafficLight_9.mpl_colors,
            )

In [ ]:
# # 计算SSE
# sse = []
# for k in range(1, 11):
#     centroids, labels = kmeans(X, k, w=weight)
#     score = silhouette_score(X, labels)
#     sse.append(score)

# # 绘制肘部图
# sns.set_style("whitegrid")
# plt.plot(range(1, 11), sse)
# plt.xlabel("Number of clusters")
# plt.ylabel("SSE")
# plt.show()

In [ ]:
score = silhouette_score(norm_X, labels)
print(f"n_clusters={n_clusters}, silhouette score={score:.4f}")

score = calinski_harabasz_score(norm_X, labels)
print(f"n_clusters={n_clusters}, Calinski-Harabasz score={score:.4f}")

In [ ]:
pca = PCA(n_components=3)
data_pca = pca.fit_transform(norm_X)

# 使用3D散点图可视化聚类结果
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data_pca[:,0], data_pca[:,1], data_pca[:,2], c=labels)
plt.show()

# # 使用3D散点图可视化聚类结果
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(X[:,0], X[:,1], X[:,2], c=labels)
# plt.show()

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2)
data_tsne = tsne.fit_transform(X)


# 使用散点图可视化聚类结果
plt.scatter(data_tsne[:,0], data_tsne[:,1], c=labels)
plt.show()

In [ ]:
# df['word freq'] = df['word freq']*100000
# df['word freq']

In [ ]:
# 并不影响lgbm的性能
# df['letter1 freq'] = df['letter1 freq'] * 10
# df['letter2 freq'] = df['letter2 freq'] * 10
# df['letter3 freq'] = df['letter3 freq'] * 10
# df['letter4 freq'] = df['letter4 freq'] * 10
# df['letter5 freq'] = df['letter5 freq'] * 10
# df['5 letters freq'] = df['5 letters freq'] * 10

In [ ]:
df["label2"] = 0

In [ ]:
df['label2'][df["difficulty factor"].apply(lambda x: x>0.8)] = 4
df['label2'][df["difficulty factor"].apply(lambda x: x>0.6 and x<=0.8)] = 3
df['label2'][df["difficulty factor"].apply(lambda x: x>0.4 and x<=0.6)] = 2
df['label2'][df["difficulty factor"].apply(lambda x: x>0.2 and x<=0.4)] = 1
df["label2"]

In [ ]:
df['label2'].apply(lambda x: x == 1).sum()

In [ ]:
df.columns

In [ ]:
y_series = df['labels']
y_series = y_series.astype('int64')
# y_series = feature_df['Accident Level']
# feature = ['Contest number',
#        'Number of  reported results', 
#        'number of repeated letters', 'number of vowels',
#        'number of common consonants', 'word commonness',
#        'number of common letters', 'LetterFreq1', 'LetterFreq2', 'LetterFreq3',
#        'LetterFreq4', 'LetterFreq5', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h',
#        'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
#        'w', 'x', 'y', 'z', 'cl_included', '1 is vowel',
#        '2 is vowel', '3 is vowel', '4 is vowel', '5 is vowel']
feature = [
       'number of repeated letters', 
       'number of common consonants', 'word commonness',
       'number of common letters']
part_category = df[feature]
# X_train, X_valid, Y_train, Y_valid = train_test_split(x_df, y_series, test_size=0.2, random_state=0, stratify=y_series)
X1_train, X1_valid, Y1_train, Y1_valid = train_test_split(part_category, y_series, test_size=0.2, random_state=0, stratify=y_series)

# lgb_train = lgb.Dataset(X_train, Y_train)
lgb1_train = lgb.Dataset(X1_train, Y1_train)
lgb1_valid = lgb.Dataset(X1_valid, Y1_valid, reference=lgb1_train)

In [ ]:
params = {
    'task' : 'train',
    'boosting' : 'gbdt',
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_error',
    'num_leaves': 100,
    'feature_fraction': 1.0,
    'bagging_fraction': 1.0,
    'bagging_freq': 0,
    'min_child_samples': 5,
    'verbose': -1,
    'feature_fraction': 0.8,
    'learning rate': 0.01,
    'lambda_l1': 7,
    'max_depth':7

}
gbm_ac1 = lgb.train(params,
            lgb1_train,
            num_boost_round=100,
            valid_sets=lgb1_valid,
            early_stopping_rounds=100)

In [ ]:
X1_train.columns

In [ ]:
a = X1_train.loc[:, ['number of repeated letters']]

In [ ]:
sns.set_theme(style="white", palette="husl")
plt.figure(dpi=1000)
ac_label = ['medium','easy','difficult']
explainer = shap.TreeExplainer(model=gbm_ac1)
# explainer = shap.TreeExplainer(model=xgbboost1)
shap_values_ac = explainer.shap_values(X=X1_train)
shap.summary_plot(shap_values=shap_values_ac, features=X1_train, feature_names=X1_train.columns, plot_type="bar", max_display=30, class_names=ac_label)

In [ ]:
# ['is_repetitive',
#        'repetitive numbers', 'number of vowels', 'number of consonants',
#        'word freq', 'number of common letter', 'letter1 freq', 'letter2 freq',
#        'letter3 freq', 'letter4 freq', 'letter5 freq', '5 letters freq', 'a',
#        'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o',
#        'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'cl_included']

In [ ]:
X_eerie = np.array([[620, 14836, 2, 4, 1, 8.564364327276781, 5, 0.09971195391262602,
                     0.09971195391262602, 0.07220355256841095, 0.05804128660585694,
                     0.09971195391262602, 0, 0, 0, 0, 3, 0, 0, 0,
                     1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
                     1, 1]])

In [ ]:
single_pred = gbm_ac1.predict(X_eerie)
label = np.argmax(single_pred)
label

In [ ]:
X_extra = np.array([df[feature].iloc[6, :]])
single_pred = gbm_ac1.predict(X_extra)
label = np.argmax(single_pred)
label

In [ ]:
X1 = df.loc[:, ['percentage_proc', '5 or more tries (X)', '4 or less tries (X)']]
scaler_X = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)

# 设置聚类个数
n_clusters = 3

# 实例化KMeans聚类器
kmeans = KMeans(n_clusters=n_clusters, random_state=0)

# 训练聚类器
kmeans.fit(X_scaled)

# 获取聚类结果
labels = kmeans.labels_

score = silhouette_score(X_scaled, kmeans.labels_)
print(f"n_clusters={n_clusters}, silhouette score={score:.4f}")

score = calinski_harabasz_score(X_scaled, kmeans.labels_)
print(f"n_clusters={n_clusters}, Calinski-Harabasz score={score:.4f}")

In [ ]:
df.columns

In [ ]:
df['1 try'] = df['1 try'] / df['percentage']
df['2 tries'] = df['2 tries'] / df['percentage']
df['3 tries'] = df['3 tries'] / df['percentage']
df['4 tries'] = df['4 tries'] / df['percentage']
df['5 tries'] = df['5 tries'] / df['percentage']
df['6 tries'] = df['6 tries'] / df['percentage']
df['7 or more tries (X)'] = df['7 or more tries (X)'] / df['percentage']
df['7 or more tries (X)']

In [ ]:
x1_ba = df["1 try"].sum()/len(df)/100
s1 = df["1 try"].apply(lambda x: (x - x1_ba)**2).sum()/(len(df)-1)/100
v1 = math.sqrt(s1)/x1_ba
v1

In [ ]:
x2_ba = df["2 tries"].sum()/len(df)/100
s2 = df["2 tries"].apply(lambda x: (x - x2_ba)**2).sum()/(len(df)-1)/100
v2 = math.sqrt(s2)/x2_ba
v2

In [ ]:
x3_ba = df["3 tries"].sum()/len(df)/100
s3 = df["3 tries"].apply(lambda x: (x - x3_ba)**2).sum()/(len(df)-1)/100
v3 = math.sqrt(s3)/x3_ba
v3

In [ ]:
x4_ba = df["4 tries"].sum()/len(df)/100
s4 = df["4 tries"].apply(lambda x: (x - x4_ba)**2).sum()/(len(df)-1)/100
v4 = math.sqrt(s4)/x4_ba
v4

In [ ]:
x5_ba = df["5 tries"].sum()/len(df)/100
s5 = df["5 tries"].apply(lambda x: (x - x5_ba)**2).sum()/(len(df)-1)/100
v5 = math.sqrt(s5)/x5_ba
v5

In [ ]:
x6_ba = df["6 tries"].sum()/len(df)/100
s6 = df["6 tries"].apply(lambda x: (x - x6_ba)**2).sum()/(len(df)-1)/100
v6 = math.sqrt(s6)/x6_ba
v6

In [ ]:
x7_ba = df["7 or more tries (X)"].sum()/len(df)/100
s7 = df["7 or more tries (X)"].apply(lambda x: (x - x7_ba)**2).sum()/(len(df)-1)/100
v7 = math.sqrt(s7)/x7_ba
v7

In [ ]:
v = [v2, v3, v4, v5, v6, v7]
v

In [ ]:
w = [0,0,0,0,0,0]
for i in range(6):
    w[i] = v[i]/sum(v)
w

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
df = pd.read_excel("./data/用于问题23的数据.xlsx")

In [ ]:
# df.columns

In [ ]:
X = df.loc[:, ['percentage_proc', '1 try',
       '2 tries', '3 tries', '4 tries', '5 tries', '6 tries',
       '7 or more tries (X)']]
# X['percentage_proc'] = X['percentage_proc']*1000000

In [ ]:
# scaler_X = StandardScaler()
scaler_X = MinMaxScaler()


X_scaled = scaler_X.fit_transform(X)

In [ ]:
# 设置聚类个数
n_clusters = 3

# 实例化KMeans聚类器
kmeans = KMeans(n_clusters=n_clusters, random_state=0)

# 训练聚类器
kmeans.fit(X)

# 获取聚类结果
labels = kmeans.labels_



score = silhouette_score(X, kmeans.labels_)
print(f"n_clusters={n_clusters}, silhouette score={score:.4f}")

score = calinski_harabasz_score(X, kmeans.labels_)
print(f"n_clusters={n_clusters}, Calinski-Harabasz score={score:.4f}")

In [ ]:
# 计算SSE
# sns.set_theme(style="white", palette="husl")
plt.figure(dpi=1000)
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, max_iter=1000, random_state=0)
    kmeans.fit(X)
    sse.append(kmeans.inertia_)

# 绘制肘部图
sns.set_style("whitegrid")
plt.plot(range(1, 11), sse)
plt.xlabel("Number of clusters")
plt.ylabel("SSE")
plt.show()

In [ ]:
sns.set_theme(style="white", palette="husl")
plt.figure(dpi=1000)
silhouette_scores = []
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, max_iter=1000, random_state=0)
    labels = kmeans.fit_predict(X)
    score = silhouette_score(X, labels)
    silhouette_scores.append(score)

# 绘制轮廓系数图
sns.set_style("whitegrid")
plt.plot(range(2, 11), silhouette_scores)
plt.xlabel("Number of clusters")
plt.ylabel("Silhouette score")
plt.show()

In [ ]:
pca = PCA(n_components=3)
data_pca = pca.fit_transform(X)

# 使用3D散点图可视化聚类结果
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data_pca[:,0], data_pca[:,1], data_pca[:,2], c=labels)
plt.show()

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

tsne = TSNE(n_components=2)
data_tsne = tsne.fit_transform(X)


# 使用散点图可视化聚类结果
plt.scatter(data_tsne[:,0], data_tsne[:,1], c=labels)
plt.show()

In [ ]:
df["label"] = labels

In [ ]:
df.loc[:, ['percentage_proc', '1 try',
       '2 tries', '3 tries', '4 tries', '5 tries', '6 tries',
       '7 or more tries (X)', 'label']].head(20)